# Reading preprocessed data

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fhvhv_covid_df = pd.read_csv('/Volumes/E/data/preprocessed_data/fhvhv_covid.csv')
fhvhv_covid_df.head()

,pickup_date,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
0,2020-09-01,34,344879,17.833136,294,34,255,278,52,9,...,45,48,73,4,58,62,24,1,18,19
1,2020-09-02,42,357969,17.718040,271,25,249,272,42,6,...,45,48,69,3,58,62,17,0,18,19
2,2020-09-03,26,391021,18.258425,274,29,250,274,41,5,...,43,46,90,11,62,67,17,4,19,20
3,2020-09-04,6,425570,19.115868,230,19,243,267,44,1,...,40,43,59,4,63,68,26,2,20,21
4,2020-09-05,0,434656,17.152276,156,11,247,269,30,3,...,40,43,34,3,63,68,24,0,22,23


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error 
from sklearn.metrics import r2_score

from numpy import absolute
from numpy import mean
from numpy import std

from statsmodels.formula.api import ols,glm
from statsmodels.api import families

# Test the significance of variables

In [8]:
# Select a sample of interest and see if all the predictors are significant
# Use Poisson distribution because trip count is discrete
# Codes are based on Lab 3's materials

fit = glm(
    formula=
    "trip_count ~ num_of_SR + avg_trip_duration + CASE_COUNT + PROBABLE_CASE_COUNT + CASE_COUNT_7DAY_AVG",
    data=fhvhv_covid_df,family=families.Poisson()).fit()

print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             trip_count   No. Observations:                  122
Model:                            GLM   Df Residuals:                      116
Model Family:                 Poisson   Df Model:                            5
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.5754e+05
Date:                Thu, 12 Aug 2021   Deviance:                   7.1328e+05
Time:                        21:32:18   Pearson chi2:                 7.24e+05
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

# Fitting ridge regression model

In [6]:
# train set: data from Sep to Nov
train_df = fhvhv_covid_df.loc[(fhvhv_covid_df['pickup_date'] >= '2020-09-01') & 
                          (fhvhv_covid_df['pickup_date'] <= '2020-11-30')]

# test_set: Dec data 
test_df = fhvhv_covid_df.loc[(fhvhv_covid_df['pickup_date'] >= '2020-12-01') & 
                          (fhvhv_covid_df['pickup_date'] <= '2020-12-31')]

In [10]:
X = train_df.drop(['pickup_date','trip_count'],axis=1).values
y = train_df['trip_count'].values
y = y.reshape(-1, 1)

# Standardize data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
scaler.fit(y)
y = scaler.transform(y)

# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

rr = Ridge(alpha=0.01)
rr.fit(X_train, y_train)
pred_train = rr.predict(X_train)

# Compute MAE, MSE, RMSE, R2 to evalute the performance of the model
print("MAE on train data: ", mean_absolute_error(y_train, pred_train))
print("MSE on train data: ", mean_squared_error(y_train, pred_train))
print("RMSE on train data: ", np.sqrt(mean_squared_error(y_train,pred_train)))
print("R2 score on train data: ", r2_score(y_train, pred_train))

rr.fit(X_test, y_test)
pred_test = rr.predict(X_test)
print("\nMAE on test data: ", mean_absolute_error(y_test, pred_test))
print("MSE on test data: ", mean_squared_error(y_test, pred_test))
print("RMSE on test data: ",np.sqrt(mean_squared_error(y_test,pred_test)))
print("R2 score on test data: ", r2_score(y_test, pred_test))

MAE on train data:  0.4330491607624971
MSE on train data:  0.31826314506250014
RMSE on train data:  0.5641481587867677
R2 score on train data:  0.6048905040839729

MAE on test data:  0.4723941749580356
MSE on test data:  0.3344889568910236
RMSE on test data:  0.578350202637661
R2 score on test data:  0.7328684149895199


# Hyperparameter tuning

In [19]:
# Grid search

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# Define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Define grid
grid = {'alpha':[0.01, 0.1, 1, 10]}

search = GridSearchCV(rr, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

results = search.fit(X, y)

print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.689
Config: {'alpha': 0.01}


# Predicting future data

In [12]:
X = test_df.drop(['pickup_date','trip_count'],axis=1).values
y = test_df['trip_count'].values
y = y.reshape(-1, 1)

# Standardize data
scaler.fit(X)
X = scaler.transform(X)
scaler.fit(y)
y = scaler.transform(y)

rr.fit(X, y)
pred_y = rr.predict(X)

# Compute MAE, MSE, RMSE, R2 to evalute the performance of the model
print("MAE on train data: ", mean_absolute_error(y, pred_y))
print("MSE on train data: ", mean_squared_error(y, pred_y))
print("RMSE on train data: ", np.sqrt(mean_squared_error(y,pred_y)))
print("R2 score on train data: ", r2_score(y, pred_y))

MAE on train data:  0.188787322094347
MSE on train data:  0.06265245680734167
RMSE on train data:  0.25030472789650154
R2 score on train data:  0.9373475431926583
